In [1]:
import os
import credentials

#set environment variable for OpenAI API Key
#os.environ["OPENAI_API_KEY"] = credentials.llm_api_key
openai_api_key = credentials.llm_api_key

## Chat Message in Langchain

Specified with a message type (System, Human, AI)

* System - Helpful background context that tell the AI what to do
* Human - Messages that are intented to represent the user
* AI - Messages that show what the AI responded with

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

In [10]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out what to eat in one short sentence"),
        HumanMessage(content="I like tomatoes, what should I eat?")
    ]
)

AIMessage(content='You can try a Caprese salad with fresh tomatoes, mozzarella, and basil.', additional_kwargs={}, example=False)

You can also pass more chat history w/ responses from the AI

In [11]:
chat(
    [
        SystemMessage(content="You are a nice AI bot that helps a user figure out where to travel in one short sentence"),
        HumanMessage(content="I like the beaches where should I go?"),
        AIMessage(content="You should go to Nice, France"),
        HumanMessage(content="What else should I do when I'm there?")
    ]
)

AIMessage(content='Besides enjoying the beaches, you should explore the charming Old Town, visit the Promenade des Anglais, and indulge in delicious Mediterranean cuisine.', additional_kwargs={}, example=False)

## Models - The interface to the AI

###  **Language Model**
A model that does text in ➡️ text out!

*Check out how I changed the model I was using from the default one to ada-001. See more models [here](https://platform.openai.com/docs/models)*

In [15]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-ada-001", openai_api_key=openai_api_key)

print(llm("What day comes after Friday?"))



Saturday


### **Chat Model**
A model that takes a series of messages and returns a message output

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

chat = ChatOpenAI(temperature=1, openai_api_key=openai_api_key)

In [17]:
chat(
    [
        SystemMessage(content="You are an unhelpful AI bot that makes a joke at whatever the user says"),
        HumanMessage(content="I would like to go to New York, how should I do this?")
    ]
)

AIMessage(content="Why don't you make friends with a wizard who can teleport you there?", additional_kwargs={}, example=False)

### **Text Embedding Model**
Change your text into a vector (a series of numbers that hold the semantic 'meaning' of your text). Mainly used when comparing two pieces of text together.

*BTW: Semantic means 'relating to meaning in language or logic.'*

In [21]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

text = "I can't belive it's Friday. I'm soo pooped! I wish it was the weekend."

In [22]:
text_embedding = embeddings.embed_query(text)
print (f"Your embedding is length {len(text_embedding)}")
print (f"Here's a sample: {text_embedding[:5]}...")

Your embedding is length 1536
Here's a sample: [-0.015920184552669525, 0.013541014865040779, -0.0064035081304609776, -0.03482698276638985, -0.024335861206054688]...


## **Prompt Template**

An object that helps create prompts based on a combination of user input, other non-static information and a fixed template string.

Think of it as an [f-string](https://realpython.com/python-f-strings/) in python but for prompts

In [23]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

# Notice "location" below, that is a placeholder for another value later
template = """
I really want to travel to {location}. What should I do there? 

Respond in one short sentence
"""

prompt = PromptTemplate(
    input_variables=["location"],
    template=template,
)

final_prompt = prompt.format(location='Rome')

print (f"Final Prompt: {final_prompt}")
print ("-----------")
print (f"LLM Output: {llm(final_prompt)}")

Final Prompt: 
I really want to travel to Rome. What should I do there?

Respond in one short sentence

-----------
LLM Output: Visit the Colosseum, the Roman Forum, the Pantheon, and the Vatican City.


### **Example Selectors**
An easy way to select from a series of examples that allow you to dynamic place in-context information into your prompt. Often used when your task is nuanced or you have a large list of examples.

Check out different types of example selectors [here](https://python.langchain.com/en/latest/modules/prompts/example_selectors.html)

If you want an overview on why examples are important (prompt engineering), check out [this video](https://www.youtube.com/watch?v=dOxUroR57xs)

In [5]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirate", "output": "ship"},
    {"input": "pilot", "output": "plane"},
    {"input": "driver", "output": "car"},
    {"input": "tree", "output": "ground"},
    {"input": "bird", "output": "nest"},
]

In [6]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(openai_api_key=openai_api_key), 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=2
)

In [8]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Give the location an item is usually found in",
    suffix="Input: {noun}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["noun"],
)

In [10]:
# Select a noun!
my_noun = "flower"

#will print nouns that are most similar to flower
print(similar_prompt.format(noun=my_noun))

Give the location an item is usually found in

Example Input: tree
Example Output: ground

Example Input: bird
Example Output: nest

Input: flower
Output:


In [11]:
llm(similar_prompt.format(noun=my_noun))

' garden'

## **Output Parsers**
A helpful way to format the output of a model. Usually used for structured output.

Two big concepts:

**1. Format Instructions** - A autogenerated prompt that tells the LLM how to format it's response based off your desired result

**2. Parser** - A method which will extract your model's text output into a desired structure (usually json)

In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [14]:
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

In [15]:
# How you would like your response structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [16]:
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
#this is the text that is input into your input prompt
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```


In [32]:
template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to michigen ya basterd!")

print(promptValue)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to michigen ya basterd!

YOUR RESPONSE:



In [33]:
llm_output = llm(promptValue)
llm_output

test_dict = output_parser.parse(llm_output)

In [34]:
type(test_dict)

dict

In [37]:
import pprint
pprint.pprint(test_dict)

{'bad_string': 'welcom to michigen ya basterd!',
 'good_string': 'Welcome to Michigan, ya bastard!'}


In [43]:
test_dict['good_string']

'Welcome to Michigan, ya bastard!'

## Indexes - Structuring documents to LLMs can work with them

### **Document Loaders**
Easy ways to import data from other sources. Shared functionality with [OpenAI Plugins](https://openai.com/blog/chatgpt-plugins) [specifically retrieval plugins](https://github.com/openai/chatgpt-retrieval-plugin)

See a [big list](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) of document loaders here. A bunch more on [Llama Index](https://llamahub.ai/) as well.

In [44]:
from langchain.document_loaders import HNLoader

loader = HNLoader("https://news.ycombinator.com/item?id=34422627")
#pass url to data loader
data = loader.load()

In [51]:
print (f"Found {len(data)} comments")
print (f"Here's a sample:\n\n{''.join([x.page_content[:150] for x in data[:2]])}")
print(f"The data type is {type(data)}")

Found 76 comments
Here's a sample:

Ozzie_osman 5 months ago  
             | next [–] 

LangChain is awesome. For people not sure what it's doing, large language models (LLMs) are very Ozzie_osman 5 months ago  
             | parent | next [–] 

Also, another library to check out is GPT Index (https://github.com/jerryjliu/gpt_index)
The data type is <class 'list'>


In [53]:
data[11]

Document(page_content='beepbooptheory 5 months ago  \n             | root | parent | next [–] \n\nAfter the dust settles, what is it really multiplying?  Especially accounting for the energy/gpus it takes.  At the very least, why not generate all the cards once, and edit as needed?I just feel like this entire concept verges on this: https://imgur.com/EiGL1Z0', metadata={'source': 'https://news.ycombinator.com/item?id=34422627', 'title': 'LangChain: Build AI apps with LLMs through composability'})

## **Text Splitters - "chunks"**

Often times your document is too long (like a book) for your LLM. You need to split it up into chunks. Text splitters help with this.

There are many ways you could split your text into chunks, experiment with [different ones](https://python.langchain.com/en/latest/modules/indexes/text_splitters.html) to see which is best for you.

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [56]:
#Load a document from memory
# This is a long document we can split up.
with open('Essay.txt') as f:
    pg_work = f.read()
    
print (f"You have {len([pg_work])} document")

You have 1 document


In [68]:
#Make chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    #a normal chunk size would be 1000-2000
    chunk_size = 150,
    chunk_overlap  = 20,
)

texts = text_splitter.create_documents([pg_work])

print (f"You have {len(texts)} documents")

You have 632 documents


In [69]:
type(texts)

list

In [70]:
print ("Preview:")
print (texts[50].page_content, "\n")
print (texts[51].page_content)

Preview:
I applied to 3 grad schools: MIT and Yale, which were renowned for AI at the time, and Harvard, which I'd visited because Rich Draves went there, and 

went there, and was also home to Bill Woods, who'd invented the type of parser I used in my SHRDLU clone. Only Harvard accepted me, so that was where


## **Retrievers**
Easy way to combine documents with language models.

There are many different types of retrievers, the most widely supported is the VectoreStoreRetriever

In [73]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('Essay.txt')
documents = loader.load()

In [74]:
type(documents)

list

In [94]:
# Get your splitter ready. Set you chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

# Initialize your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [98]:
#This takes the string and compares it to the vector and searches for similar vector within the vectorspace
docs = retriever.get_relevant_documents("what types of things did the author want to build?")

#docs = retriever.get_relevant_documents("What did the author aspire to do?")

In [99]:
#how many docs did we find?
len(docs)

4

In [100]:
#Print out first 2 documents
print("\n\n".join([x.page_content[:] for x in docs[:2]]))

In this dissatisfied state I went in 1988 to visit Rich Draves at CMU, where he was in grad school. One day I went to visit the Carnegie Institute, where I'd spent a lot of time as a kid. While looking at a painting there I realized something that might seem obvious, but was a big surprise to me. There, right on the wall, was something you could make that would last. Paintings didn't become obsolete. Some of the best ones were hundreds of years old.


And moreover this was something you could make a living doing. Not as easily as you could by writing software, of course, but I thought if you were really industrious and lived really cheaply, it had to be possible to make enough to survive. And as an artist you could be truly independent. You wouldn't have a boss, or even need to get research funding.

I started working on the application builder, Dan worked on network infrastructure, and the two undergrads worked on the first two services (images and phone calls). But about halfway thro

## **VectorStores**
Databases to store vectors. Most popular ones are [Pinecone](https://www.pinecone.io/) & [Weaviate](https://weaviate.io/). More examples on OpenAIs [retriever documentation](https://github.com/openai/chatgpt-retrieval-plugin#choosing-a-vector-database). [Chroma](https://www.trychroma.com/) & [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) are easy to work with locally.

Conceptually, think of them as tables w/ a column for embeddings (vectors) and a column for metadata.

Example

| Embedding      | Metadata |
| ----------- | ----------- |
| [-0.00015641732898075134, -0.003165106289088726, ...]      | {'date' : '1/2/23}       |
| [-0.00035465431654651654, 1.4654131651654516546, ...]   | {'date' : '1/3/23}        |

In [103]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('Essay.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [109]:
print (f"You have {len(texts)} documents")

You have 104 documents


In [112]:
#Create an embeddings list
embedding_list = embeddings.embed_documents([text.page_content for text in texts])

In [114]:
print (f"You have {len(embedding_list)} embeddings")
print (f"Here's a sample of one: {embedding_list[0][:3]}...")

You have 104 embeddings
Here's a sample of one: [0.00091109275946639, -0.011323822244166584, -0.010663658918136051]...


Your vectorstore store your embeddings (☝️) and make the easily searchable

## Memory
LLMs don't have memory by default.

Memory is a bit of a loose term. It could be as simple as remembering information you've chatted about in the past or more complicated information retrieval.

We'll keep it towards the Chat Message use case. This would be used for chat bots.

There are many types of memory, explore [the documentation](https://python.langchain.com/en/latest/modules/memory/how_to_guides.html) to see which one fits your use case.

### Chat Message History

In [116]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

history = ChatMessageHistory()

history.add_ai_message("hi!")

history.add_user_message("what is the capital of france?")

In [117]:
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False)]

In [118]:
#pass themessage history to the LLM
ai_response = chat(history.messages)
ai_response

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)

In [119]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='hi!', additional_kwargs={}, example=False),
 HumanMessage(content='what is the capital of france?', additional_kwargs={}, example=False),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={}, example=False)]

Langchain makes it easy to save the chat history so you can load it later.

### 1. Simple Sequential Chains

Easy chains where you can use the output of an LLM as an input into another. Good for breaking up tasks (and keeping your LLM focused)

In [120]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

llm = OpenAI(temperature=1, openai_api_key=openai_api_key)

In [121]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [122]:
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)

# Holds my 'location' chain
location_chain = LLMChain(llm=llm, prompt=prompt_template)

In [123]:
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)

# Holds my 'meal' chain
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

In [124]:
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)

In [127]:
review = overall_chain.run("Stockholm")



> Entering new  chain...
A classic dish from Stockholm is Kalops, which is a traditional beef and vegetable stew served with potatoes and a pickled cucumber and red beet salad.

Kalops

Ingredients:
- 2 lbs beef stew meat, cut into cubes
- 2 large onions, diced
- 2 carrots, chopped
- 2 celery stalks, chopped
- 2 cloves garlic, minced
- 2 tablespoons tomato paste
- 2 tablespoons Worcestershire sauce
- 4 cups beef broth
- 2 bay leaves
- 2 tablespoons chopped fresh parsley
- salt and pepper to taste

Instructions:
1. Heat a large saucepan or Dutch oven over medium-high heat and add the beef cubes. Sear until lightly browned.
2. Add the onions, carrots, celery, and garlic and cook until vegetables are softened.
3. Stir in the tomato paste and Worcestershire sauce and cook for a few minutes.
4. Pour in the beef broth and add the bay leaves. Bring to a boil, then reduce heat to low and simmer, covered, for 1 hour or until the beef is tender.
5. Remove from heat and season with salt and pep

### 2. Summarization Chain

Easily run through long numerous documents and get a summary. Check out [this video](https://www.youtube.com/watch?v=f9_BWhCI4Zo) for other chain types besides map-reduce

This is useful when you break up a longer piece of text into chunks, and then summarize the chunks.

In [130]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader('Essay.txt')
documents = loader.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# There is a lot of complexity hidden in this one line. I encourage you to check out the video above for more detail
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Write a concise summary of the following:


"﻿What I Worked On 



February 2021


Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep."


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU

KeyboardInterrupt: 

## Agents 🤖🤖

Official LangChain Documentation describes agents perfectly (emphasis mine):
> Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an **unknown chain** that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then **decide which, if any, of these tools to call**.


Basically you use the LLM not just for text output, but also for decision making. The coolness and power of this functionality can't be overstated enough.

Sam Altman emphasizes that the LLMs are good '[reasoning engine](https://www.youtube.com/watch?v=L_Guz73e6fw&t=867s)'. Agent take advantage of this.

### Agents

The language model that drives decision making.

More specifically, an agent takes in an input and returns a response corresponding to an action to take along with an action input. You can see different types of agents (which are better for different use cases) [here](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html).

### Tools

A 'capability' of an agent. This is an abstraction on top of a function that makes it easy for LLMs (and agents) to interact with it. Ex: Google search.

This area shares commonalities with [OpenAI plugins](https://platform.openai.com/docs/plugins/introduction).

### Toolkit

Groups of tools that your agent can select from

Let's bring them all together:

In [131]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import json

llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [132]:
serpapi_api_key=credentials.serpapi_api_key

In [133]:
#Webscraper tool
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_api_key)

In [134]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)

In [136]:
#response = agent({"input":"what was the first album of the" 
#                    "band that Natalie Bergman is a part of?"})

response = agent({"input":"what was the first album of the" 
                    "band that Erik Widman is a part of?"})



> Entering new  chain...
 I should try to find out what band Erik Widman is a part of.
Action: Search
Action Input: "Erik Widman band"
Observation: Love in October is composed of brothers Erik and Kent Widman hailing from the northern mountains of Sweden, now residing in Stockholm, Sweden and Chicago, ...
Thought: I should search for the first album of Love in October
Action: Search
Action Input: "Love in October first album"
Observation: Pontus, The Devil, and Me
Thought: I now know the final answer
Final Answer: Pontus, The Devil, and Me

> Finished chain.


In [141]:
#print(json.dumps(response["intermediate_steps"], indent=2))
#dumps(response["intermediate_steps"], indent=2)
response["intermediate_steps"]

[(AgentAction(tool='Search', tool_input='Erik Widman band', log=' I should try to find out what band Erik Widman is a part of.\nAction: Search\nAction Input: "Erik Widman band"'),
  'Love in October is composed of brothers Erik and Kent Widman hailing from the northern mountains of Sweden, now residing in Stockholm, Sweden and Chicago, ...'),
 (AgentAction(tool='Search', tool_input='Love in October first album', log=' I should search for the first album of Love in October\nAction: Search\nAction Input: "Love in October first album"'),
  'Pontus, The Devil, and Me')]